In [1]:
import os
import sys
import time as t_util
import numpy as np
import cftime
import xarray as xr
import yaml
import matplotlib.pyplot as plt


In [2]:
#Read main paths
with open('../path_main.txt', 'r') as file:    path_main  = file.read()
with open('../path_grids.txt', 'r') as file:   path_grids = file.read()
    
dir_data_Ymax = f'{path_main}Data/CMIP5/Yearmax/'
dir_scripts   = f'{path_main}Scripts/'
dir_names     = f'{path_main}Scripts/Model_lists/'
dir_regrid    = f'{path_grids}Regridding/'


## Prepare variables and parameters

In [3]:
#Define models
all_models = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models.append(eval(line[:-1]))

#Define reference period
ref_period = '1981-2010'

#Define EMT
dEMT_str = 'ESAT-3.0K'

#Define limits for European box
lon_limits_EUR = [-10, 35]
lat_limits_EUR = [30, 70]


## Regrid TXx and TNx

In [8]:
variables = ['TXx']#, 'TNx']

data_all = dict()

#Loop over variables
for variab in variables:

    dir_data = dir_data_Ymax + variab + '/'

    print(len(all_models), end=': ')

    #Loop over all models
    create = 1
    for i, model in enumerate(all_models):

        print(i, end=', ')

        #Get GCM
        if model[0] in ['CNRM-CERFACS-CNRM-CM5', 'MPI-M-MPI-ESM-LR']:
            GCM = "-".join(model[0].split('-')[2::])
        else:
            GCM = "-".join(model[0].split('-')[1::])   
            
        #Get member
        member = model[2]
        
        #Replace r3i1p1 member of EC-EARTH with a different member, as no data exits for it
        if GCM=='EC-EARTH' and member=='r3i1p1':
            member = 'r1i1p1'                   
        
        #Get file names
        f_ref = [file for file in os.listdir(dir_data) if variab in file and GCM + '_' in file and member in file and ref_period in file]
        f_app = [file for file in os.listdir(dir_data) if variab in file and GCM + '_' in file and member in file and dEMT_str in file]
        if len(f_ref)!=1:  sys.exit('File is not unique')
        if len(f_app)!=1:  sys.exit('File is not unique')

        #Read data and calculate difference
        data_ref = xr.open_dataset(dir_data + f_ref[0])
        data_app = xr.open_dataset(dir_data + f_app[0])
        data_diff = data_app.mean('time') - data_ref.mean('time')
        
        #Save in NetCDF
        fname_tmp = dir_data_Ymax + GCM + "_CMIP5-CORDEX_tmp.nc"
        data_diff.to_netcdf(fname_tmp)

        #Regrid to standard grid
        fname_grid = dir_regrid + 'grid_xy_1deg'
        fname_regrid = dir_data_Ymax + GCM + "_CMIP5-CORDEX_regridded.nc"
        os.system("cdo remapbil," + fname_grid + " " + fname_tmp + " " + fname_regrid)

        #Read regridded dataset
        data_regr = xr.open_dataset(fname_regrid)

        #Remove unnecessary variables
        vars_remove = set(data_regr.data_vars).difference([variab])
        data_regr = data_regr.drop(vars_remove)
        if 'height' in data_regr: data_regr = data_regr.drop('height')

        #Collect in one array
        if create==1:
            data_coll = data_regr
            create = 0
        else:
            data_coll = xr.concat((data_coll, data_regr), dim='model')

        #Remove temporary files
        os.remove(fname_tmp)
        os.remove(fname_regrid)

    #Add model names
    data_coll['model'] = ["_".join(model) for model in all_models]
        
    #Save in file
    fname_out = dir_data_Ymax + variab + '_CMIP5-CORDEX-ensemble_all_models_yearmax.nc'
    if os.path.exists(fname_out): os.remove(fname_out)
    data_coll.to_netcdf(fname_out)


72: 0, 1, 2, 

Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warning (cdf_set_var): Inconsistent variable definition for lat_bnds!
cdo    remapbil: Bilinear weights from gaussian (128x64) to lonlat (360x180) grid
cdo    remapbil: Processed 8192 values from 1 variable over 1 timestep [0.03s 202MB].
Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warning (cdf_set_var): Inconsistent variable definition for lat_bnds!
cdo    remapbil: Bilinear weights from gaussian (128x64) to lonlat (360x180) grid
cdo    remapbil: Processed 8192 values from 1 variable over 1 timestep [0.03s 202MB].
Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warning (cdf_set_var): Inconsistent variable definition for lat_bnds!
cdo    remapbil: Bilinear weights from gaussian (256x128) to lonlat (360x180) grid
cdo    remapbil: Processed 32768 values from 1 variable over 1 timestep [0.04s 202MB].
Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warn

3, 4, 5, 

Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warning (cdf_set_var): Inconsistent variable definition for lat_bnds!
cdo    remapbil: Bilinear weights from gaussian (256x128) to lonlat (360x180) grid
cdo    remapbil: Processed 32768 values from 1 variable over 1 timestep [0.04s 202MB].
Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warning (cdf_set_var): Inconsistent variable definition for lat_bnds!
cdo    remapbil: Bilinear weights from gaussian (256x128) to lonlat (360x180) grid
cdo    remapbil: Processed 32768 values from 1 variable over 1 timestep [0.04s 202MB].
Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warning (cdf_set_var): Inconsistent variable definition for lat_bnds!
cdo    remapbil: Bilinear weights from gaussian (256x128) to lonlat (360x180) grid


6, 7, 8, 

...

Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warning (cdf_set_var): Inconsistent variable definition for lat_bnds!
cdo    remapbil: Bilinear weights from lonlat (144x96) to lonlat (360x180) grid
cdo    remapbil: Processed 13824 values from 1 variable over 1 timestep [0.02s 243MB].
Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warning (cdf_set_var): Inconsistent variable definition for lat_bnds!
cdo    remapbil: Bilinear weights from lonlat (144x96) to lonlat (360x180) grid
cdo    remapbil: Processed 13824 values from 1 variable over 1 timestep [0.02s 243MB].
Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warning (cdf_set_var): Inconsistent variable definition for lat_bnds!
cdo    remapbil: Bilinear weights from lonlat (144x96) to lonlat (360x180) grid
cdo    remapbil: Processed 13824 values from 1 variable over 1 timestep [0.02s 244MB].


68, 69, 70, 

Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warning (cdf_set_var): Inconsistent variable definition for lat_bnds!
cdo    remapbil: Bilinear weights from lonlat (144x96) to lonlat (360x180) grid
cdo    remapbil: Processed 13824 values from 1 variable over 1 timestep [0.02s 244MB].
Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warning (cdf_set_var): Inconsistent variable definition for lat_bnds!
cdo    remapbil: Bilinear weights from lonlat (144x96) to lonlat (360x180) grid
cdo    remapbil: Processed 13824 values from 1 variable over 1 timestep [0.02s 245MB].
Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warning (cdf_set_var): Inconsistent variable definition for lat_bnds!
cdo    remapbil: Bilinear weights from lonlat (144x96) to lonlat (360x180) grid
cdo    remapbil: Processed 13824 values from 1 variable over 1 timestep [0.02s 245MB].


71, 

Warning (cdf_set_var): Inconsistent variable definition for lon_bnds!
Warning (cdf_set_var): Inconsistent variable definition for lat_bnds!
cdo    remapbil: Bilinear weights from lonlat (144x96) to lonlat (360x180) grid
cdo    remapbil: Processed 13824 values from 1 variable over 1 timestep [0.02s 246MB].
